In [44]:
import geopandas as gpd
gpd.options.io_engine = "pyogrio"
import requests
import os
import zipfile
import tempfile

## Download Vector Data
To illustrate the visualization of vector data, we download 4-digit postcodes from Switzerland and store them as ESRI Shapefile in the local directory `./data/`.

More information about the data can be found on the [homepage](https://www.swisstopo.admin.ch/en/official-directory-of-towns-and-cities) of Federal Office of Topography swisstopo.



In [46]:
# Download the ZIP file from the URL
url = "https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/ortschaftenverzeichnis_plz/ortschaftenverzeichnis_plz_2056.shp.zip"
data_dir = "./data/"

os.makedirs(data_dir, exist_ok=True)

with tempfile.TemporaryDirectory() as tmpdirname:
    # Define the path for the temporary ZIP file
    temp_zip_path = os.path.join(tmpdirname, "temp.zip")

    # Download the ZIP file and write to the temporary file
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Raise an error for bad status codes
        with open(temp_zip_path, "wb") as f:
            f.write(response.content)
            
    # Extract the ZIP file to the specified directory
    with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

## Data Preparation
We modify the data using [geopandas](https://geopandas.org/en/stable/). We reproject the data from the local Swiss coordinate sytsem [LV95](https://en.wikipedia.org/wiki/Swiss_coordinate_system) to the [World Geodetic System 84](https://en.wikipedia.org/wiki/World_Geodetic_System#WGS84) and store it as [GeoJson](https://en.wikipedia.org/wiki/GeoJSON). This format is chosen to later facilitate the creation of vector tiles.

In [47]:
gdf_plz = gpd.read_file("./data/AMTOVZ_SHP_LV95/AMTOVZ_ZIP.shp")
gdf_plz = gdf_plz[["ZIP4", "geometry"]].to_crs("EPSG:4326")
gdf_plz.to_file("data/swiss_zip.geojson", driver="GeoJSON") 

## Create Vector Tileset
To create the vector tileset, we use the comand line tool [tippecanoe](https://github.com/felt/tippecanoe?tab=readme-ov-file), which was initially created by Mapbox and now maintained as open source project.

In [48]:
!tippecanoe \
    -z15 \
    -o data/swiss_zip.pmtiles \
    --coalesce-densest-as-needed \
    --extend-zooms-if-still-dropping \
    --detect-shared-borders \
    data/swiss_zip.geojson

For layer 0, using name "swiss_zip"
4073 features, 11166412 bytes of geometry and attributes, 19170 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
  99.9%  15/17090/11488  


Above cell could be written something like this:
```
!ogr2ogr \
  -f GeoJSON \
  -t_srs EPSG:4326 \
  ./data/swiss_zip.geojson \
  ./data/AMTOVZ_SHP_LV95/AMTOVZ_ZIP.shp
```

For below cell there is no Python package:

Above cell could be rewriten something like this:

```
!curl -L -o ./data/temp.zip \
    https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/ortschaftenverzeichnis_plz/
    ortschaftenverzeichnis_plz_2056.shp.zip
!unzip ./data/temp.zip -d ./data/
```